<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/lluvia_estaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y estandarización de datasets de lluvia (Quito)

## Objetivo
Unificar y depurar las series de precipitación **horaria, diaria y mensual** de estaciones dentro del **DMQ (Quito)** para el periodo **2019 a 2025**, aplicando controles de calidad y completitud, y exportar **datasets limpios y consistentes** (diario y mensual) listos para análisis de inundaciones y para su integración con el notebook principal.

---

## Contenido
1. Importación de Librerías
2. Carga de datos
3. Filtro geográfico (solo Quito)
4. Normalización y limpieza
5. Manejo de series acumuladas (si aplica)
6. Agregaciones
7. Control de calidad (QA)
8. Unificación de granularidades
9. Consolidación por estación y por zona
10. Exportación




## 1. **Importación de Librerías**

In [2]:
# Importación de librerías

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## 2. **Carga de datos**

In [3]:
# Lluvias diarias
precipitaciones_diarias = pd.read_csv(r'/content/P09-Iñaquito_INAMHI_Precipitación-Diario.csv')

precipitaciones_diarias.head()

,fecha,valor,completo_mediciones,completo_umbral
0,2014/07/01,0.0,100.0,100.0
1,2014/07/02,0.0,100.0,100.0
2,2014/07/03,0.0,100.0,100.0
3,2014/07/04,0.0,100.0,100.0
4,2014/07/05,0.0,100.0,100.0


In [6]:
precipitaciones_diarias2 = pd.read_csv(r'/content/tableDownload (4).csv')

precipitaciones_diarias2.head()

,Fecha,RN
0,2019-11-06 23:50,173.2
1,2019-11-07 23:00,0.0
2,2019-11-08 23:00,0.0
3,2019-11-09 23:00,0.2
4,2019-11-10 23:00,0.4


In [4]:
# Lluvias horaria

precipitaciones_horaria = pd.read_csv(r'/content/P09-Iñaquito_INAMHI_Precipitación-Horario.csv')

precipitaciones_horaria.head()

,fecha,valor,completo_mediciones
0,2014/07/01 00:00:00,0.0,100.0
1,2014/07/01 01:00:00,0.0,100.0
2,2014/07/01 02:00:00,0.0,100.0
3,2014/07/01 03:00:00,0.0,100.0
4,2014/07/01 04:00:00,0.0,100.0


In [5]:
# Lluvia Mensual

precipitaciones_mensual = pd.read_csv(r'/content/P09-Iñaquito_INAMHI_Precipitación-Mensual.csv')

precipitaciones_mensual.head()

,fecha,valor,completo_mediciones,completo_umbral
0,2014/07,0.2,100.0,100.0
1,2014/08,2.9,100.0,100.0
2,2014/09,58.5,100.0,100.0
3,2014/10,110.9,100.0,100.0
4,2014/11,59.7,100.0,100.0


Renombrar a nombres estándar

In [10]:
df = precipitaciones_diarias.copy()

# --- renombrar a nombres estándar
df = df.rename(columns={
    "fecha": "fecha",
    "valor": "p_mm_dia",                    # precipitación diaria en mm
    "completo_mediciones": "completo_med",
    "completo_umbral": "completo_umbral"
})

df.head()

,fecha,p_mm_dia,completo_med,completo_umbral
0,2014/07/01,0.0,100.0,100.0
1,2014/07/02,0.0,100.0,100.0
2,2014/07/03,0.0,100.0,100.0
3,2014/07/04,0.0,100.0,100.0
4,2014/07/05,0.0,100.0,100.0


In [15]:
# Manejo de duplicados

duplicados_fecha = df.duplicated(keep=False)
# df = df.drop_duplicates(subset=["fecha"]).sort_values("fecha")
conteo_duplicados_fecha = duplicados_fecha.sum()

print(f'conteo de fechas duplicadas: {conteo_duplicados_fecha}')

conteo de fechas duplicadas: 0


In [16]:
# QA sencillo de completitud diaria
# muchos recursos de INAMHI traen 100 = 100% completitud; si no están esas columnas, marcamos True.
if {"completo_med","completo_umbral"}.issubset(df.columns):
    # aceptamos el día si completo_med >= 80% del umbral (regla simple)
    df["qa_diario_ok"] = (df["completo_med"] >= 0.8 * df["completo_umbral"])
else:
    df["qa_diario_ok"] = True

